In [1]:
import numpy as np
import pandas as pd
import sympy as sym
import re

In [2]:
#Functions used by the algorithm

def R(k,N,fcn):
    """Returns the value of the operator R at the mode k, depending on which resonant mode N and 
    differential operator L are being considered. R(k) is defined by L(k)-L(1), and L satisfies L(N)=L(1).
    
    Args:
        k (int) : the mode at which to calculate R. 
        N (int) : The value of the resonant mode.
        fcn (callable) : The differential operator L to be used within R.
        
    Returns:
        float : The value of R for the given parameters.
    """
    
    R = fcn(k,N) - fcn(1,N)
    return R

def RecurringCoeff(N,fcn):
    """Returns the value of the recurring coefficient for a given resonant mode N and differential operator L.
    The coefficient is given by 1/2R(2) - 1/R(N-1) - 1/R(N+1). 
    
    Args:
        N (int) : The value of the resonant mode.
        fcn (callable) : The differential operator L to be used within R.
        
    Returns:
        float : The value of the recurring coefficient.
    """
    
    A = 1/(2*R(2,N,fcn)) - 1/R(N-1,N,fcn) - 1/R(N+1,N,fcn)
    return A

def RecurringCoeff_3A(b1,fcn):
    """Returns the coefficients of b_{n-2} in the first solvability condition when N = 3, for a given b1 and differential
    operator L.
    
    Args:
        b1 (float) : the value of the first resonant coefficient
        fcn (callable) : The differential operator L to be used within R.
        
    Returns:
        float : The value of the recurring coefficient.
    """
    return 2*b1*(1/R(2,3,fcn) + 1/R(4,3,fcn))+3/(2*R(2,3,fcn))

def RecurringCoeff_3B(s2,b1,fcn):
    """Returns the coefficients of b_{n-2} in the second solvability condition when N = 3, for a given b1 and differential
    operator L.
    
    Args:
        b1 (float) : the value of the first resonant coefficient
        fcn (callable) : The differential operator L to be used within R.
        
    Returns:
        float : The value of the recurring coefficient.
    """
    return s2 + 3*b1**2/(2*R(6,3,fcn)) + 1/R(2,3,fcn) + 1/R(4,3,fcn)

def high_mode(l,N):
    """Returns the highest mode found at order l for a given resonant mode N. 
    The highest mode is given by l + 2 floor(l/(N-2)).
    
    Args:
        l (int) : the order at which to the determine highest mode present.
        N (int) : The value of the resonant mode.
        
    Returns:
        int : The highest mode found at order l.
    """
    return 2*int(l/(N-2))+l

def delta(k,j):
    """The Kronecker delta function. Returns 1 if the two inputs are the same, and 0 otherwise.
    
    Args:
        k (float) : the first argument to the Kronecker delta function.
        j (float) : the second argument to the Kronecker delta function.
        
    Returns:
        int : 1 if k and j are equal, and 0 otherwise.
    """
    if k == j:
        return 1
    else:
        return 0

In [3]:
#Functions defining the various differential operators considered.

def L_Kawahara(k,N):
    """The differential operator L for the operator defining the Kawahara equation. Returns the value of the operator for
    the mode k, depending on the given resonant mode N.
    
    Args:
        k (int) : The mode at which to calculate L.
        N (int) : The value of the resonant mode.
        
    Returns:
        float : The value of L at the given mode.
    """
    L = k**2 - (k**4/(N**2 + 1))
    return L

def L_Benjamin(k,N):
    """The differential operator L for the operator defining the Benjamin equation. Returns the value of the operator for
    the mode k, depending on the given resonant mode N.
    
    Args:
        k (int) : The mode at which to calculate L.
        N (int) : The value of the resonant mode.
        
    Returns:
        float : The value of L at the given mode.
    """
    L = abs(k) - (k**2)/(N+1)
    return L

def L_Akers(k,N):
    """The differential operator L for the operator defining the Akers-Milewski equation. Returns the value of the operator for
    the mode k, depending on the given resonant mode N.
    
    Args:
        k (int) : The mode at which to calculate L.
        N (int) : The value of the resonant mode.
        
    Returns:
        float : The value of L at the given mode.
    """
    L = 1/abs(k) + abs(k)/N
    return L

def L_Whitham(k,N):
    """The differential operator L for the operator defining the Deep-Water Whitham equation. Returns the value of the operator for
    the mode k, depending on the given resonant mode N.
    
    Args:
        k (int) : The mode at which to calculate L.
        N (int) : The value of the resonant mode.
        
    Returns:
        float : The value of L at the given mode.
    """
    L = np.sqrt(L_Akers(k,N))
    return L

def L_hydroelastic(k,N):
    """The differential operator L for the operator defining the hydroelastic equation. Returns the value of the operator for
    the mode k, depending on the given resonant mode N.
    
    Args:
        k (int) : The mode at which to calculate L.
        N (int) : The value of the resonant mode.
        
    Returns:
        float : The value of L at the given mode.
    """
    L = np.sqrt(1/abs(k)+(abs(k**3))/(N**3+N**2+N))
    return L


In [4]:
#Define important variables

N = 3 #Resonant mode
L = L_Kawahara #Differential operator in question (choose from above functions)
max_order = 20 #Max order to calculate (Full solution will only be given up to max_order - 2)
max_mode = high_mode(max_order,N) #The highest mode to be found at any order
solnum = 0 #Choose which solution to use for N = 3

In [5]:
#Initialize solution arrays
beta = np.zeros([max_order+1,3*max_mode]) #Non-resonant mode coefficients (Extra space given to prevent IndexErrors)
s = np.zeros(max_order+1) #Wavespeed corrections
b = np.zeros(max_order+1) #Resonant coefficients

In [14]:
#Initial Conditions
for L in [L_Kawahara,L_Benjamin,L_Akers,L_Whitham,L_Hydroelastic]:
    for solnum in range(3):
        if N == 3:

            #Determine s2 and b1
            s2,b1 = sym.symbols('s_2,b_1') #Initialize symbols

            #Create system of equations
            eq1 = sym.Eq(s2 + 1/(2*R(2,3,L)) + (3/(2*R(2,3,L)))*b1 + (1/R(2,3,L) + 1/R(4,3,L))*(b1**2),0)
            eq2 = sym.Eq(1/(2*R(2,3,L)) + s2*b1 + (1/R(2,3,L) + 1/R(4,3,L))*b1 + 1/(2*R(6,3,L))*(b1**3),0)

            #Solve for s2 and b1
            sols = np.complex128(sym.solve([eq1,eq2],(s2,b1)))

            #Assign desired solution (eliminate imaginary portions which are only present due to floating point error)
            s[2],b[1] = sols[solnum].real

            #Initial Conditions
            beta[2,2] = (2*b[1]+1)/(2*R(2,N,L))
            beta[2,4] = (b[1])/(R(4,N,L))
            beta[2,6] = (b[1]**2)/(2*R(6,N,L))
            beta[3,5] = (b[1]*beta[2,2]+beta[2,4]+beta[2,6])/(R(5,N,L))
            beta[3,7] = (b[1]*beta[2,4]+beta[2,6])/(R(7,N,L))
            beta[3,9] =  (b[1]*beta[2,6])/(R(9,N,L))

        else:
            beta[2,2] = 1/(2*R(2,N,L))
            s[2] = -1/(2*R(2,N,L))
            beta[3,3] = 1/(2*R(2,N,L)*R(3,N,L))

In [15]:
if N == 3:
    #Iterate through orders 4 to max_order
    for n in np.arange(4,max_order + 1).astype(int):
        print(n)
        
        #If n odd, solve for s[n-1] and b[n-2]
        if n % 2 == 1:
            print('Calculating wavespeed correction and resonant coefficient')
            
            #Define matrix denoting left-hand side of solvability conditions
            A = np.array([[1,RecurringCoeff_3A(b[1],L)],[b[1],RecurringCoeff_3B(s[2],b[1],L)]])

            #Initialize right-hand sums
            S1 = 0 #First solvability condition
            S2 = 0 #Second solvability condition 

            #Add initial terms
            S1 += b[1]*(beta[n-1,2]+beta[n-1,4])+beta[n-1,2]
            S2 += beta[n-1,2] + beta[n-1,4] + b[1]*beta[n-1,6]

            #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
            for l in np.arange(1,(n-3)/2+1).astype(int):

                
                if 2*l != (n-3): #Following terms not present if 2l = n-3 (would then contain b[n-2])
                    
                    #To first solvability condition, add resonant coefficient times neighbouring modes (2 and 4)
                    S1 += b[2*l+1]*(beta[n-2*l-1,2] + beta[n-2*l-1,4])
                    
                    #To first solvability condition, add resonant coefficient times wavespeed correction and 6th mode
                    S2 += b[2*l+1]*(s[n-2*l-1] + beta[n-2*l-1,6])

                #For even orders, sum over even modes
                for k in np.arange(1,3*l+1).astype(int):
                    
                    if k != 1: #If k = 1 (i.e. 2nd mode), then there is no contribution to first solvability condition 
                        S1 += 0.5*beta[2*l,2*k]*((1-delta(k,2))*beta[n-2*l,2*k-1] + beta[n-2*l,2*k+1])
                    
                    S2 += 0.5*beta[2*l,2*k]*((1-delta(abs(2*k-3),1)-delta(k,3))*beta[n-2*l,abs(2*k-3)]+beta[n-2*l,2*k+3])
                
                #For odd orders, sum over odd modes
                for k in np.arange(2,3*l+2).astype(int):
                    S1 += 0.5*beta[2*l+1,2*k+1]*(beta[n-2*l-1,2*k]+beta[n-2*l-1,2*k+2])
                    S2 += 0.5*beta[2*l+1,2*k+1]*(beta[n-2*l-1,2*k+1-3]+beta[n-2*l-1,2*k+1+3])
   
            #Define vector with components of the sums just calculated
            B = np.array([-S1,-S2])

            #s[n-2] and b[n-1] are solution x to system given by A.x = B -> therefore x = inv(A).B
            sol = np.dot(np.linalg.inv(A),B)
            
            s[n-1],b[n-2] = sol

            #Update beta values from previous order
            beta[n-1,2] = beta[n-1,2] + b[n-2]/R(2,3,L)
            beta[n-1,4] = beta[n-1,4] + b[n-2]/R(4,3,L)
            beta[n-1,6] = beta[n-1,6] + b[1]*b[n-2]/R(6,3,L)

        #Calculate non-resonant coefficients
        print('Calculating non-resonant coefficients')
        
        #If n is even, only even modes must be considered
        if n % 2 == 0:
            for j in np.arange(2,3*n+1,2).astype(int):
                
                #Initialize sum
                beta_sum = 0
                
                #Add neighbouring modes
                if j not in [2,4]: #beta[n-1,j-1] does not exist if j = 2 or j = 4
                    beta_sum += beta[n-1,j-1]
                
                if j != 2: #beta[n-1,j+1] does not exist if j = 2
                    beta_sum += beta[n-1,j+1]
                    
                #Combine b[1] with modes 3 away from j from previous order
                beta_sum += b[1]*((1-delta(j,2)-delta(j,4)-delta(j,6))*beta[n-1,j-3]+beta[n-1,j+3])

                #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                for l in np.arange(1,(n-2)/2+1).astype(int):

                    #Add wavespeed correction times jth mode
                    beta_sum += s[n-2*l]*beta[2*l,j]

                    #For even orders, sum over even modes
                    for k in np.arange(1,3*l+1).astype(int):
                            beta_sum += 0.5*beta[2*l,2*k]*((1-delta(2*k,j))*beta[n-2*l,abs(j-2*k)]+beta[n-2*l,j+2*k])
                            
                    if 2*l != (n-2): #Following terms not included if 2l = n-2
                        
                        #Add resonant coefficient times modes 3 away from j
                        beta_sum += b[2*l+1]*((1-delta(j,2)-delta(j,4)-delta(j,6))*beta[n-2*l-1,abs(j-3)]+beta[n-2*l-1,j+3]+delta(j,6)*0.5*b[n-2*l-1])

                        #For odd orders, sum over odd modes
                        for k in np.arange(2,3*l+2).astype(int):
                            beta_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(abs(2*k+1-j),1)-delta(abs(2*k+1-j),3))*beta[n-2*l-1,abs(2*k+1-j)]+beta[n-2*l-1,j+2*k+1])
                
                #Set value as sum divided by R value at that j
                beta[n,j] = beta_sum/R(j,3,L)
        
        #If n is odd, only odd modes must be considered
        else:
            for j in np.arange(5,3*n+1,2).astype(int):
                
                #Initialize sum
                beta_sum = 0

                #Add neighbouring modes
                beta_sum += beta[n-1,j-1]
                beta_sum += beta[n-1,j+1]
                
                #Combine b[1] with modes 3 away from j from previous order
                beta_sum += b[1]*(beta[n-1,j-3]+beta[n-1,j+3])

                #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                for l in np.arange(1,(n-3)/2+1).astype(int):

                    #Add wavespeed correction times jth mode
                    beta_sum += s[n-2*l-1]*beta[2*l+1,j]

                    #Add resonant coefficient times modes 3 away from j
                    beta_sum += b[2*l+1]*(beta[n-2*l-1,j-3]+beta[n-2*l-1,j+3])

                    #For even orders, sum over even modes
                    for k in np.arange(1,3*l+1).astype(int):
                            beta_sum += 0.5*beta[2*l,2*k]*((1-delta(abs(2*k-j),1)-delta(abs(2*k-j),3))*beta[n-2*l,abs(j-2*k)]+beta[n-2*l,j+2*k])
                    
                    #For odd orders, sum over odd modes
                    for k in np.arange(2,3*l+2).astype(int):
                            beta_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(2*k+1,j))*beta[n-2*l-1,abs(2*k+1-j)]+beta[n-2*l-1,j+2*k+1])
                
                #Set value as sum divided by R value at that j
                beta[n,j] = beta_sum/R(j,3,L)

else:
    #Iterate through orders 4 to max_order
    for n in range(4,max_order+1):
        print(n)
        
        #Calculate wavespeed correction
        print('Calculating wavespeed correction')

        #Initialize sum
        s_sum = 0

        #Proceed only if n is odd
        if n%2 == 1:

            #Add second mode from previous order
            s_sum += beta[n-1,2]

            #Check parity of N
            if N % 2 == 0:

                #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                for l in np.arange(1,(n-3)/2+1).astype(int):

                    #Add resonant coefficient times the neighboring modes N + 1 and N - 1
                    s_sum += b[2*l]*(beta[n-2*l,N-1]+beta[n-2*l,N+1])

                    #For even orders, sum over even modes
                    for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                        if k != N/2:
                            s_sum += 0.5*beta[2*l,2*k]*(beta[n-2*l,2*k+1]+(1-delta(k,1))*beta[n-2*l,2*k-1])

                    #For odd order, sum over odd modes
                    for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                        s_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(2*k+2,N))*beta[n-2*l-1,2*k+2]+(1-delta(2*k,N))*beta[n-2*l-1,2*k])
            else:
                #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                for l in np.arange(1,(n-3)/2+1).astype(int):

                    #Add resonant coefficient times the neighboring modes N + 1 and N - 1
                    s_sum += b[2*l+1]*(beta[n-2*l-1,N-1]+beta[n-2*l-1,N+1])

                    #For even orders, sum over even modes
                    for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                        s_sum += 0.5*beta[2*l,2*k]*((1-delta(2*k,N-1))*beta[n-2*l,2*k+1]+(1-delta(2*k,N+1))*beta[n-2*l,2*k-1])

                    #For odd order, sum over odd modes
                    for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                        if k != (N-1)/2:
                            s_sum += 0.5*beta[2*l+1,2*k+1]*(beta[n-2*l-1,2*k+2]+beta[n-2*l-1,2*k])

        #Set wavespeed corrections as negative of above sum
        s[n-1] = -s_sum

        #Calculate resonant coefficient
        print('Calculating resonant coefficient')


        #Initialize sum
        b_sum = 0

        #Check parity of N and n - do not proceed if not the same.
        if N % 2 == 0:
            if n % 2 == 0:

                #Add neighboring modes N + 1 and N - 1 from previous order
                b_sum += beta[n-1,N+1] + beta[n-1,N-1] #Note current values in beta array are beta bar

                #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                for l in np.arange(1, (n-2)/2+1).astype(int):
                    if 2*l != n-2: #Following terms are not included when 2l = n-2

                        #Add resonant coefficient times wavespeed correction and 2N mode.
                        b_sum += b[2*l]*(s[n-2*l] + beta[n-2*l,2*N])

                        #For odd order, sum over odd modes
                        for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                            b_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(abs(N-2*k-1),1))*beta[n-2*l-1,abs(N-2*k-1)]+beta[n-2*l-1,N+2*k+1])

                    #For even order, sum over even modes
                    for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                        if k != N/2:
                            b_sum += 0.5*beta[2*l,2*k]*(beta[n-2*l,abs(N-2*k)]+beta[n-2*l,N+2*k])
        else:
            if n % 2 == 1:

                #Add neighboring modes N + 1 and N - 1 from previous order
                b_sum += beta[n-1,N+1] + beta[n-1,N-1] #Note current values in beta array are beta bar

                #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                for l in np.arange(1, (n-3)/2+1).astype(int):
                    if (2*l+1) != n-2: #Following terms are not included when 2l = n-2

                        #Add resonant coefficient times wavespeed correction and 2N mode.
                        b_sum += b[2*l+1]*(s[n-2*l-1] + beta[n-2*l-1,2*N])

                    #For even order, sum over even modes
                    for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                        b_sum += 0.5*beta[2*l,2*k]*((1-delta(abs(N-2*k),1))*beta[n-2*l,abs(N-2*k)]+beta[n-2*l,N+2*k])

                    #For odd order, sum over odd modes
                    for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                        if k != (N-1)/2:
                            b_sum += 0.5*beta[2*l+1,2*k+1]*(beta[n-2*l-1,abs(N-2*k-1)]+beta[n-2*l-1,N+2*k+1])

        #Set value as sum over recurring coefficient
        b[n-2] = b_sum/RecurringCoeff(N,L)

        #Update neighboring modes N + 1 and N - 1 from previous order 
        beta[n-1,N-1] = beta[n-1,N-1] + b[n-2]/R(N-1,N,L)
        beta[n-1,N+1] = beta[n-1,N+1] + b[n-2]/R(N+1,N,L)

        #Calculate non-resonant coefficients
        print('Calculating non-resonant coefficients')

        #If n is even, only even modes must be considered
        if n % 2 == 0:

            #Iterate over even modes
            for j in np.arange(2, high_mode(n,N)+1,2).astype(int):

                #Initialize sum
                beta_sum = 0

                #Check parity of N
                if N % 2 == 0:
                    if j != N: #j cannot be N

                        #Add neighbouring modes j - 1 and j + 1 from previous order
                        beta_sum += (1-delta(j,2))*beta[n-1,j-1]+beta[n-1,j+1]

                        #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                        for l in np.arange(1,(n-2)/2+1).astype(int):

                            #Add wavespeed correction times jth mode
                            beta_sum += s[n-2*l]*beta[2*l,j]

                            #Add resonant coefficient times modes which are N away from j 
                            beta_sum += b[2*l]*((1-delta(j,2*N))*beta[n-2*l,abs(N-j)]+beta[n-2*l,N+j])

                            #If j is 2*N, add resonant coefficients are multiplied together
                            if j == 2*N:
                                beta_sum += 0.5*b[2*l]*b[n-2*l]

                            #For even orders, sum over even modes
                            for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                                if k != N/2:
                                    beta_sum += 0.5*beta[2*l,2*k]*((1-delta(2*k,j)-delta(abs(2*k-j),N))*beta[n-2*l,abs(2*k-j)]+(1-delta(2*k+j,N))*beta[n-2*l,2*k+j])

                            if 2*l != n-2: #Following terms are not included when 2l = n-2

                                #For odd orders, sum over odd modes
                                for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                                    beta_sum += 0.5*(beta[2*l+1,2*k+1]*((1-delta(abs(2*k+1-j),1))*beta[n-2*l-1,abs(2*k+1-j)]+beta[n-2*l-1,2*k+1+j]))

                elif N % 2 == 1:

                    #Add neighbouring modes j - 1 and j + 1 from previous order
                    beta_sum += (1-delta(j,2)-delta(j,N+1))*beta[n-1,j-1] + (1-delta(j,N-1))*beta[n-1,j+1]

                    #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                    for l in np.arange(1,(n-2)/2+1).astype(int):

                        #Add wavespeed correction times jth mode
                        beta_sum += s[n-2*l]*beta[2*l,j]

                        #For even orders, sum over even modes
                        for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                            beta_sum += 0.5*beta[2*l,2*k]*((1-delta(2*k,j))*beta[n-2*l,abs(2*k-j)]+beta[n-2*l,2*k+j])

                        if 2*l != n-2: #Following terms are not included when 2l = n-2

                            #Add resonant coefficient times modes which are N away from j 
                            beta_sum += b[2*l+1]*((1-delta(abs(N-j),1)-delta(j,2*N))*beta[n-2*l-1,abs(N-j)]+beta[n-2*l-1,N+j]+0.5*delta(j,2*N)*b[n-2*l-1])

                            #For odd orders, sum over odd modes
                            for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                                if k != (N-1)/2:
                                    beta_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(abs(2*k+1-j),1)-delta(abs(2*k+1-j),N))*beta[n-2*l-1,abs(2*k+1-j)]+(1-delta(2*k+1+j,N))*beta[n-2*l-1,2*k+1+j])

                #Set value as sum divided by R value at that j
                beta[n,j] = beta_sum/R(j,N,L)

        #If n is odd, only odd modes must be considered
        elif n % 2 == 1:

            #Iterate over odd modes
            for j in np.arange(3, high_mode(n,N)+1,2).astype(int):

                #Initialize sum
                beta_sum = 0

                #Check parity of N
                if N % 2 == 0:

                    #Add neighbouring modes j - 1 and j + 1 from previous order
                    beta_sum += (1-delta(j,N+1))*beta[n-1,j-1] + (1-delta(j,N-1))*beta[n-1,j+1]

                    #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                    for l in np.arange(1,(n-3)/2+1).astype(int):

                        #Add wavespeed correction times jth mode
                        beta_sum += s[n-2*l-1]*beta[2*l+1,j]

                        #Add resonant coefficient times modes which are N away from j 
                        beta_sum += b[2*l]*((1-delta(abs(N-j),1))*beta[n-2*l,abs(N-j)]+beta[n-2*l,N+j])

                        #For even orders, sum over even modes
                        for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                            if k != N/2:
                                beta_sum += 0.5*beta[2*l,2*k]*((1-delta(abs(2*k-j),1))*beta[n-2*l,abs(2*k-j)]+beta[n-2*l,2*k+j])

                        #For odd orders, sum over odd modes
                        for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                            beta_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(2*k+1,j)-delta(abs(2*k+1-j),N))*beta[n-2*l-1,abs(2*k+1-j)]+(1-delta(2*k+1+j,N))*beta[n-2*l-1,2*k+1+j])

                elif N % 2 == 1:
                    if j != N: #j cannot be N

                        #Add neighbouring modes j - 1 and j + 1 from previous order
                        beta_sum += beta[n-1,j-1] + beta[n-1,j+1]

                        #Iterate over l, combine order 2l with order n-2l and order 2l+1 with order n - 2l - 1
                        for l in np.arange(1,(n-3)/2+1).astype(int):

                            #Add wavespeed correction times jth mode
                            beta_sum += s[n-2*l-1]*beta[2*l+1,j]

                            #Add resonant coefficient times modes which are N away from j 
                            beta_sum += b[2*l+1]*(beta[n-2*l-1,abs(j-N)]+beta[n-2*l-1,j+N])

                            #For even orders, sum over only even modes
                            for k in np.arange(1,high_mode(2*l,N)/2+1).astype(int):
                                beta_sum += 0.5*beta[2*l,2*k]*((1-delta(abs(2*k-j),1)-delta(abs(2*k-j),N))*beta[n-2*l,abs(2*k-j)]+(1-delta(2*k+j,N))*beta[n-2*l,2*k+j])

                            #For odd orders, sum over only modes
                            for k in np.arange(1,(high_mode(2*l+1,N)-1)/2+1).astype(int):
                                beta_sum += 0.5*beta[2*l+1,2*k+1]*((1-delta(2*k+1,j))*beta[n-2*l-1,abs(2*k+1-j)]+beta[n-2*l-1,2*k+1+j])

                #Set value as sum divided by R value at that j
                beta[n,j] = beta_sum/R(j,N,L)

#Truncate extra zeroes from beta array
beta = beta[:,:max_mode + 1]

4
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
5
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
6
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
7
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
8
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
9
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
10
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
11
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
12
Calculating wavespeed correction
Calculating resonant coefficient
Calculating non-resonant coefficients
13
Calculating wavespeed correction
Calcula